In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
# Disable truncation for pandas output
pd.set_option('display.max_colwidth', None)

In [29]:
import os
import pandas as pd

# Define the folder containing the dataset
folder_path = "/kaggle/input/include"

# Lists to store video paths and labels
video_files = []
labels = []

# Read the folder structure
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".MOV"):  # Check for video files
            video_path = os.path.join(root, file)
            video_files.append(video_path)

            # Extract the label from the folder name
            # Assuming the folder names are like "Adjectives_1of8"
            label = os.path.basename(root).replace('_', ' ')
#             print(label)
            labels.append(label.split(" ")[-1])

# Create a DataFrame
df = pd.DataFrame({"video_name": video_files, "tag": labels})

# Display the DataFrame
# print(df.head())


In [30]:
xdf = df[0:100]

In [33]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
xdf['tag'] = label_encoder.fit_transform(xdf['tag'])
xdf

/tmp/ipykernel_34/2762686943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf['tag'] = label_encoder.fit_transform(xdf['tag'])


,video_name,tag
0,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV,2
1,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5138.MOV,2
2,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5297.MOV,2
3,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5136.MOV,2
4,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_9408.MOV,2
...,...,...
95,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_9250.MOV,0
96,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_9333.MOV,0
97,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_9332.MOV,0
98,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_5139.MOV,0


In [34]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

In [8]:
!pip install -q pytorchvideo transformers[sentencepiece] evaluate
!pip install accelerate -U
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [35]:
import av
# import datasets
# from datasets import load_dataset, DatasetDict,  Audio
import pandas as pd
import os
import glob
import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score
from transformers import AutoImageProcessor, VideoMAEModel,  AdamW
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [36]:
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

In [37]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.array(np.stack([x.to_ndarray(format="rgb24") for x in frames]))

In [38]:
def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


In [39]:
def get_image_processor_inputs(file_path, image_processor):
    container = av.open(file_path)
    indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)
    inputs = image_processor(list(video), return_tensors="pt")
    
#     video_tensor = inputs['pixel_values']
#     if video_tensor.dim() == 4:  # If the shape is (num_frames, num_channels, height, width)
#         video_tensor = video_tensor.unsqueeze(0) 
    return inputs
#     return video_tensor

In [14]:
# t = VideoClassificationDataset("/kaggle/input/include", train_df,  image_processor).__getitem__(0)
# t[0].shape

In [40]:
class VideoClassificationDataset(Dataset):
    def __init__(self, root_dir, data_df, image_processor):
        self.data = data_df.dropna()
        self.root_dir = root_dir
        self.image_processor = image_processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        video_path = self.data.iloc[index]['video_name']

#         video_path = os.path.join(self.root_dir, image_path)

        video_encodings = get_image_processor_inputs(video_path, image_processor).pixel_values
#         video_encodings = get_image_processor_inputs(video_path, image_processor)
        labels = np.array(self.data.iloc[index]["tag"])

        return video_encodings, labels
#         return  {"pixel_values": video_encodings, "labels": torch.tensor(labels)}

In [41]:

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
video_encoder = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
# video_encoder = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base", num_labels=train_df['tag'].nunique())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
train_dataset = VideoClassificationDataset("/kaggle/input/include", train_df,  image_processor)
test_dataset = VideoClassificationDataset("/kaggle/input/include",  test_df, image_processor)
val_dataset = VideoClassificationDataset("/kaggle/input/include",  val_df, image_processor)

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [18]:
# for i in train_loader:
#     print(i[0])
#     print(type(i))
#     break

In [43]:
class VideoClassifier(nn.Module):
    def __init__(self, num_labels, video_encoder):
        super(VideoClassifier, self).__init__()
        self.video_encoder = video_encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.video_encoder.config.hidden_size, 2048),
            nn.ReLU(), 
            
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_labels)
        )

    def forward(self, video_features):
        outputs = self.video_encoder(video_features)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits


In [44]:
num_labels = train_df['tag'].nunique()

model = VideoClassifier(num_labels, video_encoder).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
criterion = nn.CrossEntropyLoss()


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
# from typing import List, Union
# import torch
# from transformers import AutoImageProcessor

# class CustomDataCollator:
#     def __init__(self, image_processor: AutoImageProcessor):
#         self.image_processor = image_processor

#     def __call__(self, batch):
#         video_encodings, labels = zip(*batch)

#         # Assuming video_encodings contains batches of videos as tensors
#         # For demonstration, assuming video_encodings are already in the correct tensor format
#         video_encodings_stacked = torch.stack(video_encodings, dim=0)

#         # Process video encodings using image_processor (tokenizer)
#         processed_inputs = self.image_processor(list(video_encodings_stacked), return_tensors="pt")

#         return {
#             "input_ids": processed_inputs.input_ids,
#             "pixel_values": processed_inputs.pixel_values,
#             "labels": torch.tensor(labels),
#         }


In [22]:
# from transformers import TrainingArguments, Trainer
# # Training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy"
# )

# # Custom metrics function
# from sklearn.metrics import accuracy_score

# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     return {"accuracy": accuracy_score(p.label_ids, preds)}
# custom_collator = CustomDataCollator(image_processor)

# # Trainer
# trainer = Trainer(
#     model=video_encoder,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=image_processor,
#     compute_metrics=compute_metrics,
#     data_collator=custom_collator
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# trainer.evaluate(test_dataset)

In [48]:

# Define the training function
def train(model, train_loader, val_loader, optimizer,  criterion, device, num_epochs):

    best_accuracy = 0.0

    for epoch in range(num_epochs):

        model.train()

        for i, batch in enumerate(train_loader):

            video_features, labels = batch

            if video_features.shape[0] == 1:
              video_features = video_features.squeeze(0)
            else:
              video_features = video_features.squeeze()

            video_features = video_features.to(device)

            labels = labels.view(-1)
            labels = labels.to(device)

            optimizer.zero_grad()

            logits = model(video_features)

            loss = criterion(logits, labels)
            loss.backward()

            optimizer.step()

            if (i+1) % 8 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item() :.4f}')
                train_loss = 0.0

        val_loss, val_accuracy, val_f1, _ , _ = evaluate(model, val_loader, device)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt')

        print("========================================================================================")
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}, best_accuracy: {best_accuracy:.4f}')
        print("========================================================================================")



In [49]:

def evaluate(model, data_loader,  device):

    print("evaluate started")
    all_labels = []
    all_preds = []
    total_loss = 0.0

    with torch.no_grad():

        for i, batch in enumerate(data_loader):

          video_features, labels = batch

          if video_features.shape[0] == 1:

            video_features = video_features.squeeze(0)
          else:
            video_features = video_features.squeeze()

          video_features = video_features.to(device)

          labels = labels.view(-1)
          labels = labels.to(device)

          logits = model(video_features)
          loss = criterion(logits, labels)
          total_loss += loss.item()

          _, preds = torch.max(logits, 1)

          all_labels.append(labels.cpu().numpy())
          all_preds.append(preds.cpu().numpy())


    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)

    loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return loss, accuracy, f1, all_labels, all_preds



In [25]:
# torch.cuda.reset_max_memory_allocated()

In [50]:
torch.cuda.empty_cache()
video_encoder.gradient_checkpointing_enable()
num_epochs = 5
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)

Epoch 1/5, Batch 8/196, Train Loss: 5.5798
Epoch 1/5, Batch 16/196, Train Loss: 5.5555
Epoch 1/5, Batch 24/196, Train Loss: 5.5621
Epoch 1/5, Batch 32/196, Train Loss: 5.5607
Epoch 1/5, Batch 40/196, Train Loss: 5.5480
Epoch 1/5, Batch 48/196, Train Loss: 5.5605
Epoch 1/5, Batch 56/196, Train Loss: 5.5402
Epoch 1/5, Batch 64/196, Train Loss: 5.5424
Epoch 1/5, Batch 72/196, Train Loss: 5.5427
Epoch 1/5, Batch 80/196, Train Loss: 5.5505
Epoch 1/5, Batch 88/196, Train Loss: 5.5521
Epoch 1/5, Batch 96/196, Train Loss: 5.5387
Epoch 1/5, Batch 104/196, Train Loss: 5.5437
Epoch 1/5, Batch 112/196, Train Loss: 5.5295
Epoch 1/5, Batch 120/196, Train Loss: 5.5185
Epoch 1/5, Batch 128/196, Train Loss: 5.5356
Epoch 1/5, Batch 136/196, Train Loss: 5.5527
Epoch 1/5, Batch 144/196, Train Loss: 5.4868
Epoch 1/5, Batch 152/196, Train Loss: 5.4777
Epoch 1/5, Batch 160/196, Train Loss: 5.5161
Epoch 1/5, Batch 168/196, Train Loss: 5.4501
Epoch 1/5, Batch 176/196, Train Loss: 5.4738
Epoch 1/5, Batch 184/19

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch 1/5, Val Loss: 5.4357, Val Accuracy: 0.0254, Val F1: 0.0014, best_accuracy: 0.0254
Epoch 2/5, Batch 8/196, Train Loss: 5.3618



KeyboardInterrupt



In [ ]:
# import os
# import numpy as np
# import torch
# import pytorchvideo
# import pytorchvideo.data
# from torch.utils.data import DataLoader, Dataset
# from torchvision.transforms import Compose, Lambda, Resize, RandomHorizontalFlip, RandomCrop
# from pytorchvideo.transforms import ApplyTransformToKey, Normalize, UniformTemporalSubsample
# from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification, TrainingArguments, Trainer
# import evaluate
# from sklearn.model_selection import train_test_split

# ######################################################################################
# ######################          DEFINE PATHS HERE      ###############################
# ######################################################################################

# # Path to INCLUDE dataset
# dataset_root_path = '/kaggle/input/include/Adjectives_1of8/Adjectives'  # Replace with the actual path to your dataset
# model_ckpt = "MCG-NJU/videomae-base"
# batch_size = 8

# ######################################################################################
# ######################################################################################
# ######################################################################################

# # Load all video file paths for determining classes
# all_video_file_paths = [os.path.join(dp, f) for dp, dn, filenames in os.walk(dataset_root_path) for f in filenames if f.endswith(".MOV")]

# # Extract class labels and create mapping dictionaries
# class_labels = sorted({str(path).split("/")[-2] for path in all_video_file_paths})
# label2id = {label: i for i, label in enumerate(class_labels)}
# id2label = {i: label for label, i in label2id.items()}

# # Load pre-trained image processor and model
# image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
# model = VideoMAEForVideoClassification.from_pretrained(
#     model_ckpt,
#     label2id=label2id,
#     id2label=id2label,
#     ignore_mismatched_sizes=True  # Necessary if the model expects a different input size
# )

# # Extract necessary parameters from the image processor
# mean = image_processor.image_mean
# std = image_processor.image_std

# # Determine resizing strategy
# if "shortest_edge" in image_processor.size:
#     resize_to = (image_processor.size["shortest_edge"], image_processor.size["shortest_edge"])
# else:
#     resize_to = (224, 224)  # Use a default size if specific sizes are not available

# num_frames_to_sample = model.config.num_frames
# sample_rate = 4
# fps = 30
# clip_duration = num_frames_to_sample * sample_rate / fps

# # Define transformations
# train_transform = Compose([
#     ApplyTransformToKey(
#         key="video",
#         transform=Compose([
#             UniformTemporalSubsample(num_frames_to_sample),
#             Lambda(lambda x: x / 255.0),
#             Normalize(mean, std),
#             RandomCrop(resize_to),
#             RandomHorizontalFlip(p=0.5),
#         ]),
#     ),
# ])

# val_transform = Compose([
#     ApplyTransformToKey(
#         key="video",
#         transform=Compose([
#             UniformTemporalSubsample(num_frames_to_sample),
#             Lambda(lambda x: x / 255.0),
#             Normalize(mean, std),
#             Resize(resize_to),
#         ]),
#     ),
# ])

# # Custom dataset class to handle INCLUDE dataset's `.MOV` files
# class SignLanguageDataset(Dataset):
#     def __init__(self, root_dir, transform=None, clip_duration=clip_duration):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.clip_duration = clip_duration
#         self.video_paths, self.labels = self._load_dataset()
#         self.label_map = {label: idx for idx, label in enumerate(sorted(set(self.labels)))}

#     def _load_dataset(self):
#         video_paths = []
#         labels = []
#         for root, dirs, files in os.walk(self.root_dir):
#             for file in files:
#                 if file.endswith(".MOV"):
#                     label = os.path.basename(root)
#                     video_paths.append(os.path.join(root, file))
#                     labels.append(label)
#         return video_paths, labels

#     def __len__(self):
#         return len(self.video_paths)

#     def __getitem__(self, idx):
#         video_path = self.video_paths[idx]
#         label = self.label_map[self.labels[idx]]

#         video = pytorchvideo.data.encoded_video.EncodedVideo.from_path(video_path)
#         frames, _ = video.get_clip(0, self.clip_duration)
        
#         if self.transform:
#             frames = self.transform({"video": frames})["video"]  # Apply transformations

#         return {"video": frames, "label": label}

# # Instantiate dataset
# dataset = SignLanguageDataset(dataset_root_path, transform=train_transform)

# # Calculate the number of samples
# n_samples = len(dataset)

# # Adjust train_size and test_size based on the number of samples
# train_size = int(n_samples * 0.8)  # 80% for training
# test_size = n_samples - train_size  # Remaining for testing

# # Split dataset into training and validation sets
# train_dataset, val_dataset = train_test_split(dataset, train_size=train_size, test_size=test_size, random_state=42)

# # Create DataLoaders
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# # Define metrics computation
# metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     predictions = np.argmax(eval_pred.predictions, axis=1)
#     return metric.compute(predictions=predictions, references=eval_pred.label_ids)

# # Data collation function
# def collate_fn(examples):
#     pixel_values = torch.stack([example["video"].permute(1, 0, 2, 3) for example in examples])
#     labels = torch.tensor([example["label"] for example in examples])
#     return {"pixel_values": pixel_values, "labels": labels}

# # Set up Trainer with appropriate training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=5,  # Modify this based on your dataset size
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     logging_dir='./logs',
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_loader.dataset,
#     eval_dataset=val_loader.dataset,
#     tokenizer=image_processor,
#     compute_metrics=compute_metrics,
#     data_collator=collate_fn,
# )

# # Start training
# trainer.train()


In [ ]:
# import os
# from shutil import copy2
# from sklearn.model_selection import train_test_split


# def videoprocessing(path, mp4path):
#     for root, _, file in os.walk(path):
#         for filename in file:
#             if filename.endswith(".MOV"):
#                 print(root.split('/')[-1])
#                 print(root)
#                 os.makedirs(os.path.join(mp4path, root.split('/')[-1], "mp4"), exist_ok=True)
#                 base_filename, _ = os.path.splitext(filename)
#                 output_filename = f"{base_filename}.mp4"
#                 input_filepath = os.path.join(root, filename)
#                 # os.mkdir(os.path.join(root, "mp4")
#                 output_filepath = os.path.join(mp4path, root.split('/')[-1], output_filename)
#                 os.system(f"ffmpeg -i {input_filepath} {output_filepath}")
#                 print(f"Converted {filename} to {output_filename}")

# def split_video_data(mp4folder, output_folder, train_size=0.8, val_size=0.1, test_size=0.1):
#     all_videos = {}  # Dictionary to store videos per category
#     for category in os.listdir(mp4folder):
#         category_path = os.path.join(mp4folder, category)
#         if os.path.isdir(category_path):
#             category_videos = []
#             for root, _, files in os.walk(category_path):
#                 for file in files:

#                     video_path = os.path.join(root, file)
#                     category_videos.append(video_path)
#             all_videos[category] = category_videos

#     # Print the number of videos found in each category for debugging
#     print("Number of videos found per category:")
#     for category, videos in all_videos.items():
#         print(f"{category}: {len(videos)}")

#     if not all_videos:
#         raise ValueError("No video files found in the input folder.")

#     # Split videos within each category
#     for category, videos in all_videos.items():
#         train_videos, test_val_videos = train_test_split(videos, train_size=train_size, random_state=42)
#         val_videos, test_videos = train_test_split(test_val_videos, test_size=test_size / (test_size + val_size), random_state=42)

#         os.makedirs(os.path.join(output_folder, 'train', category), exist_ok=True)
#         os.makedirs(os.path.join(output_folder, 'val', category), exist_ok=True)
#         os.makedirs(os.path.join(output_folder, 'test', category), exist_ok=True)

#         for folder, video_list in [('train', train_videos), ('val', val_videos), ('test', test_videos)]:
#             for video in video_list:
#                 output_path = os.path.join(output_folder, folder, category, os.path.basename(video))
#                 copy2(video, output_path)

# if __name__ == "__main__":
#     input_folder = "/kaggle/input/include/Adjectives_1of8"  
#     mp4path = "/kaggle/working/" # Dumb fucking logic, useless intermediate path for mp4s
#     output_folder = "'/kaggle/working/ouput'"
#     videoprocessing(input_folder, mp4path)
#     split_video_data(mp4path, output_folder)
#     # TODO: add argparse

In [ ]:
# !rm -rf /kaggle/working/*

###FINETUNING WITH NUMPY ARRAY

DIRECTLY FINETUNING WITH VIDEOS